In [1]:
import os

current_dir_path = os.getcwd()
current_file_path = os.path.abspath(current_dir_path)
current_dir_path = os.path.dirname(current_file_path)
parent_dir_path = os.path.dirname(current_dir_path)

os.chdir(current_dir_path + '\\ultils')

import pickle
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from tdamapper.core import MapperAlgorithm
from tdamapper.cover import CubicalCover
from tdamapper.clustering import FailSafeClustering
from tdamapper.plot import MapperLayoutInteractive, MapperLayoutStatic

from sklearn import metrics

from functions import *
from chi import *
from regressionP import *
from models import *

In [2]:
data1 = pd.read_csv("../Data/NPA_TMA2_1.csv", low_memory=False)[:-2]
data2 = pd.read_csv("../Data/NPA_TMA2_2.csv", low_memory=False)[:-2]
data3 = pd.read_csv("../Data/NPA_TMA2_3.csv", low_memory=False)[:-2]
data4 = pd.read_csv("../Data/NPA_TMA2_4_new.csv", low_memory=False)[:-2]
data5 = pd.read_csv("../Data/NPA_TMA2_5.csv", low_memory=False)[:-2]
data6 = pd.read_csv("../Data/NPA_TMA2_6_new.csv", low_memory=False)[:-2]
data7 = pd.read_csv("../Data/NPA_TMA2_7.csv", low_memory=False)[:-2]
data8 = pd.read_csv("../Data/NPA_TMA2_8.csv", low_memory=False)[:-2]
data9 = pd.read_csv("../Data/NPA_TMA2_9.csv", low_memory=False)[:-2]
data10 = pd.read_csv("../Data/NPA_TMA2_10.csv", low_memory=False)[:-2]

dataA2 = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10], ignore_index=True)

dataA1 = pd.read_csv("../Data/NPA_TMA1_V3.csv")[:-2]

In [3]:
print(dataA1.shape)
print(dataA2.shape)

print(dataA2.columns)

(3631, 51)
(664995, 51)
Index(['發生年度', '發生月份', '發生日期', '發生時間', '事故類別名稱', '處理單位名稱警局層', '發生地點', '天候名稱',
       '光線名稱', '道路類別-第1當事者-名稱', '速限-第1當事者', '道路型態大類別名稱', '道路型態子類別名稱',
       '事故位置大類別名稱', '事故位置子類別名稱', '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
       '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱', '號誌-號誌種類名稱', '號誌-號誌動作名稱',
       '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分向設施子類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
       '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱', '事故類型及型態大類別名稱',
       '事故類型及型態子類別名稱', '肇因研判大類別名稱-主要', '肇因研判子類別名稱-主要', '死亡受傷人數', '當事者順位',
       '當事者區分-類別-大類別名稱-車種', '當事者區分-類別-子類別名稱-車種', '當事者屬-性-別名稱', '當事者事故發生時年齡',
       '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱', '當事者行動狀態大類別名稱', '當事者行動狀態子類別名稱',
       '車輛撞擊部位大類別名稱-最初', '車輛撞擊部位子類別名稱-最初', '車輛撞擊部位大類別名稱-其他', '車輛撞擊部位子類別名稱-其他',
       '肇因研判大類別名稱-個別', '肇因研判子類別名稱-個別', '肇事逃逸類別名稱-是否肇逃', '經度', '緯度'],
      dtype='object')


In [4]:
select_lst = [
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱', 
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱', 
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱', 
    '道路類別-第1當事者-名稱', '速限-第1當事者', '道路型態大類別名稱', '事故類型及型態大類別名稱', '事故位置大類別名稱',
    '死亡受傷人數',
]

def preprocess(input_data, select_lst):
    
    sample_data = input_data[input_data['當事者順位'] == 1].reset_index(drop=True, inplace=False)
    sample_data = sample_data[sample_data['發生月份'] < 4]
    dataA = sample_data[select_lst]
    
    # 將資料分出死亡和受傷，合併到原本的資料後去除多餘的死亡受傷人數
    death_injury_data = split_death_injury(dataA['死亡受傷人數'])
    dist_df = pd.concat([dataA, death_injury_data], axis=1)
    dist_df.drop(columns=['死亡受傷人數'], inplace=True)
    
    return dist_df

dist_dfA1 = preprocess(dataA1, select_lst)
dist_dfA2 = preprocess(dataA2, select_lst)

dist_dfA1.head()

,路面狀況-路面鋪裝名稱,路面狀況-路面狀態名稱,路面狀況-路面缺陷名稱,道路障礙-障礙物名稱,道路障礙-視距品質名稱,道路障礙-視距名稱,號誌-號誌種類名稱,號誌-號誌動作名稱,車道劃分設施-分向設施大類別名稱,車道劃分設施-分道設施-快車道或一般車道間名稱,車道劃分設施-分道設施-快慢車道間名稱,車道劃分設施-分道設施-路面邊線名稱,道路類別-第1當事者-名稱,速限-第1當事者,道路型態大類別名稱,事故類型及型態大類別名稱,事故位置大類別名稱,死亡,受傷
0,柏油,乾燥,無缺陷,無障礙物,無遮蔽物,良好,無號誌,無號誌,中央分向島,車道線(附標記),快慢車道分隔線,有,省道,40.0,單路部分,車輛本身,其他,1,0
1,柏油,乾燥,無缺陷,無障礙物,無遮蔽物,良好,閃光號誌,正常,中央分向島,未繪設車道線,快慢車道分隔線,有,市區道路,30.0,交岔路,車與車,交叉路口,1,1
2,柏油,乾燥,無缺陷,無障礙物,無遮蔽物,良好,無號誌,無號誌,雙向禁止超車線,車道線(無標記),快慢車道分隔線,有,縣道,50.0,單路部分,車輛本身,路段,1,0
3,柏油,乾燥,無缺陷,無障礙物,無遮蔽物,良好,閃光號誌,正常,無,未繪設車道線,未繪設快慢車道分隔線,無,市區道路,30.0,交岔路,車與車,交叉路口,1,1
4,柏油,乾燥,無缺陷,無障礙物,無遮蔽物,良好,無號誌,無號誌,行車分向線,未繪設車道線,未繪設快慢車道分隔線,有,省道,50.0,單路部分,車輛本身,路段,1,0


In [5]:
rbind_data = pd.concat([dist_dfA1, dist_dfA2], axis=0, ignore_index=True)
assert rbind_data.shape[0] == dist_dfA1.shape[0] + dist_dfA2.shape[0]

dummy_data = pd.get_dummies(rbind_data)

scaler = MinMaxScaler()
columns_to_normalize = ['速限-第1當事者']
dummy_data[columns_to_normalize] = scaler.fit_transform(dummy_data[columns_to_normalize])

print(dummy_data.shape)
dummy_data.head()

(96846, 78)


,速限-第1當事者,死亡,受傷,路面狀況-路面鋪裝名稱_其他鋪裝,路面狀況-路面鋪裝名稱_柏油,路面狀況-路面鋪裝名稱_水泥,路面狀況-路面鋪裝名稱_無鋪裝,路面狀況-路面鋪裝名稱_碎石,路面狀況-路面狀態名稱_乾燥,路面狀況-路面狀態名稱_冰雪,...,道路型態大類別名稱_圓環廣場,道路型態大類別名稱_平交道,事故類型及型態大類別名稱_人與車,事故類型及型態大類別名稱_平交道事故,事故類型及型態大類別名稱_車與車,事故類型及型態大類別名稱_車輛本身,事故位置大類別名稱_交叉路口,事故位置大類別名稱_交流道,事故位置大類別名稱_其他,事故位置大類別名稱_路段
0,0.079208,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
1,0.059406,1,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
2,0.099010,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
3,0.059406,1,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
4,0.099010,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [36]:
data_to_map = dummy_data.drop(['死亡', '受傷'], axis=1)
mapper_numpy = data_to_map.to_numpy()

pca = PCA(n_components=10)
lens1 = pca.fit_transform(mapper_numpy)

# 查看每個主成分保留的變異量比例
explained_variance_ratio  = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
print(cumulative_variance_ratio)
print(sum(explained_variance_ratio))
print(explained_variance_ratio)

[0.27239782 0.42265716 0.50953113 0.58348504 0.63607678 0.68182045
 0.72215348 0.76001574 0.79035559 0.81948407]
0.819484073435112
[0.27239782 0.15025934 0.08687396 0.07395392 0.05259173 0.04574367
 0.04033303 0.03786226 0.03033985 0.02912848]


In [22]:
# import umap

# reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
# embedding = reducer.fit_transform(mapper_numpy)

# # 將嵌入結果轉換為 DataFrame
# embedding_df = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2'])
# embedding_df['label'] = dummy_data['受傷']

# # 視覺化結果
# plt.figure(figsize=(10, 8))
# sns.scatterplot(
#     x='UMAP1', y='UMAP2',
#     hue='label',  # 以 label 分類顏色
#     palette='tab10',  # 色盤
#     data=embedding_df,
#     legend='full',
#     alpha=0.8
# )
# plt.title('UMAP Projection of the Dataset')
# plt.show()

In [7]:
# plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, marker='o')
# plt.xlabel('Number of Principal Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.title('Elbow Method for PCA')
# plt.grid(True)
# plt.show()

In [43]:
start_time = time.time()

mapper_algo1 = MapperAlgorithm(
    cover = CubicalCover(
        n_intervals = 3,
        overlap_frac = 0.4
    ),
    clustering = FailSafeClustering(
        clustering = AgglomerativeClustering(10),
        verbose = False)
)

mapper_graph1 = mapper_algo1.fit_transform(mapper_numpy, lens1)

end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)

157.85603189468384


In [44]:
mapper_plot1 = MapperLayoutInteractive(
    mapper_graph1,
    colors = dummy_data[['死亡']].to_numpy(),
    cmap = 'jet',
    agg = np.nanmean,
    # agg = most_frequent_nonan,
    dim = 3,
    iterations = 30,
    seed = 6,
    width = 800,
    height = 500)

In [46]:
# fig_mean1 = mapper_plot1.plot()
# fig_mean1.show(config={'scrollZoom': True})

In [12]:
# import pickle

# with open('CalculatedData/道路V2_dummy.pkl', 'wb') as f:
#     pickle.dump(mapper_graph1, f)